# Annotated Types and Type Variables

Technically this is not Pydantic specific, but it is a very commonly used technique for defining types in Pydantic models.

Suppose we want to define a bounded list of elements that contains at at most 10 elements.

We've seen how to do this using the `Field` class:

In [1]:
from typing import Annotated

from pydantic import BaseModel, Field, ValidationError

In [2]:
class Model(BaseModel):
    elements: list[int] = Field(default=[], max_length=10)

In [3]:
m = Model(elements=[1, 2, 3])
m

Model(elements=[1, 2, 3])

In [4]:
try:
    Model(elements = [1, ] * 20)
except ValidationError as ex:
    print(ex)

1 validation error for Model
elements
  List should have at most 10 items after validation, not 20 [type=too_long, input_value=[1, 1, 1, 1, 1, 1, 1, 1, ... 1, 1, 1, 1, 1, 1, 1, 1], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/too_long


We could, for reusability, define this as an annotated type instead:

In [5]:
BoundedListInt = Annotated[list[int], Field(max_length=10)]

And now we can re-use it in any of our models:

In [6]:
class Model(BaseModel):
    field_1: BoundedListInt = []
    field_2: BoundedListInt = []

So that's great, but this annotated type is for lists of integers only.

What if we wanted the same thing for lists of floats?

In [7]:
BoundedListFloat = Annotated[list[float], Field(max_length=10)]

And for the same thing but for strings?

In [8]:
BoundedListString = Annotated[list[str], Field(max_length=10)]

And so on...

As you can see, the only thing that changes is that the type in `list[]` changes, from `int`, to `float` to `str`. And if we wanted other types, we'd have keep creating these types that are otherwise identical.

We could use the `Any` type in our annotation - but doing so we would lose the type validation Pydantic has to offer (since it would now accept lists that contained any type, including lists with mixed types) - which is probably not something we want.

In [9]:
from typing import Any

BoundedList = Annotated[list[Any], Field(max_length=10)]

This is where the concept of a type variables comes into play.

Think about how we actually defined a list of integers, or a list of strings...

We used the type `list`, then square brackets and the element type inside the square brackets, e.g. `list[int]`.

That element type could have been any type we wanted, even a custom class.

In other words, the type defined inside those square brackets is actually variable.

That's the concept of type variables, implemented in Python using `TypeVar`.

Let's see how to do this for our own types.

First we need to define a type variable - we can name it whatever we want, and here I'll just use `T`.

In [10]:
from typing import TypeVar

In [11]:
T = TypeVar('T')

We now have a type variable, and we can use it when we create our annotated type - we'll use that variable instead of using `int` or `float`.

In [12]:
BoundedList = Annotated[list[T], Field(max_length=10)]

Now, to create a bounded list of a specific type, we just have to specify the value for the type variable when we use the annotated type:

In [13]:
BoundedList[int]

typing.Annotated[list[int], FieldInfo(annotation=NoneType, required=True, metadata=[MaxLen(max_length=10)])]

In [14]:
BoundedList[str]

typing.Annotated[list[str], FieldInfo(annotation=NoneType, required=True, metadata=[MaxLen(max_length=10)])]

And we can use this in our models:

In [15]:
class Model(BaseModel):
    integers: BoundedList[int] = []
    strings: BoundedList[str] = []

In [16]:
Model()

Model(integers=[], strings=[])

In [17]:
Model(integers=[1.0, 2.0], strings=["abc", "def"])

Model(integers=[1, 2], strings=['abc', 'def'])

And the field list elements are specifically typed:

In [18]:
try:
    Model(integers=[0.5])
except ValidationError as ex:
    print(ex)

1 validation error for Model
integers.0
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=0.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.5/v/int_from_float


There's a lot more you can do with type variables, but this is not a course on Python typing - and although I did specify that type hinting was a pre-requisite to this course, i wanted you to be aware of this technique specifically as you will find it useful when trying to define re-usable Pydantic types.